In [23]:
from src.llm.api_wrappers import OpenaiAPI
import pypdf
from PyPDF2 import PdfReader
import io
import re

In [24]:
api = OpenaiAPI(model_name='gpt-4o-mini')

In [28]:
def get_pdf_text(file_path):

    with open(file_path, "rb") as fp:
        # Load the file in memory if the filesystem is not the default one to avoid
        # issues with pypdf
        stream = io.BytesIO(fp.read())
    
        # Create a PDF object
        pdf = pypdf.PdfReader(stream)
    
        # Get the number of pages in the PDF document
        num_pages = len(pdf.pages)
    
        # Join text extracted from each page
        
        text = "\n".join(
            pdf.pages[page].extract_text() for page in range(num_pages)
        )
        
        text = re.sub(r'\s+', ' ', text)
        # Remove spaces between letters (e.g., "r o c k e t" -> "rocket")
        text = re.sub(r'(\w)\s+(\w)', r'\1\2', text)
    
    return text


def get_pdf_text2(file_path):
    reader = PdfReader(file_path)
    text = "\n".join(
        reader.pages[page].extract_text(space_width=500) for page in range(len(reader.pages))
    )
    # replace \n with space
    text = text.replace("\n", " ")
    return text


def create_chunks(text, chunk_size=800):
    sentences = text.split(".")
    chunks = []
    current_chunk = ""
    while sentences:
        sentence = sentences.pop(0) + "."
        current_chunk += sentence
        if len(current_chunk) > chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
            
    chunks.append(current_chunk)
    return chunks

In [31]:
path = "./data/rocket_liquid_prop/"
# file = path + "Surface_Roughness.pdf"
file = path + "part_1.pdf"
# path = "./data/financial/"
# file = path + "lyft_2021.pdf"



text = get_pdf_text2(file)
chunks = create_chunks(text, 300)

In [32]:
print(chunks)

['Fundamental  Concepts of  Liquid-Propellant  Rocket EnginesAlessandro de Iaco VerisSpringer Aerospace Technology Springer Aerospace Technology Series Editors Sergio De Rosa, DII, University of Naples Federico II, NAPOLI, Italy Yao Zheng, School of Aeronautics and Astronautics, Zhejiang University, Hangzhou, Zhejiang, China The series explores the technology and the science related to the aircraft and spacecraft including concept, design, assembly, control and maintenance.', 'Fundamental  Concepts of  Liquid-Propellant  Rocket EnginesAlessandro de Iaco VerisSpringer Aerospace Technology Springer Aerospace Technology Series Editors Sergio De Rosa, DII, University of Naples Federico II, NAPOLI, Italy Yao Zheng, School of Aeronautics and Astronautics, Zhejiang University, Hangzhou, Zhejiang, China The series explores the technology and the science related to the aircraft and spacecraft including concept, design, assembly, control and maintenance. The topics cover aircraft, missiles, spac

In [5]:
system_prompt = """\
You are a Teacher/Professor. Your task is to create \
a question for an upcoming quiz/examination. \
The questions should be technical in nature. \
Restrict the questions to the context information provided. \

Only use the context and no prior knowledge to generate the questions. \
Base your question on information specific to the context. \
Only ask questions that can be clearly answered ONLY using the provided information. \
The question should be at most 30 words long. Avoid using the same phrases as in the \
context as this could give hints to the answer. \
"""

In [6]:

# chunk = chunks[14]

# print(len(chunks))
# response = api.get_text_response(chunk, system_prompt)

num_async = 5
for i in range(1, num_async):
    question_chunks = chunks[i:i+num_async]
    responses = api.get_async_text_response(question_chunks, system_prompt)
    print(responses)
    break

for response in responses:
    print(response.get_as_str())


RuntimeError: Cannot run the event loop while another loop is running